<a href="https://colab.research.google.com/github/ev1025/GA_LOG_ANALYST/blob/main/GA%EC%A0%84%EC%B2%98%EB%A6%AC(Hits%EC%A0%9C%EC%99%B8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install ipywidgets # tqdm

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'


In [ ]:
import pandas as pd
import numpy as np
from pandas import json_normalize
import json
import os
import csv
import ast
from tqdm.notebook import tqdm
from tqdm import tqdm
from datetime import datetime

In [ ]:
# train_v2가 있는 경로로 이동
os.chdir('C:\\Users\\ENVY\\Desktop\\GA')

## 데이터 가져오기

In [ ]:
pd.set_option('display.max_columns',None)

In [ ]:
# 데이터 보기
pre_df = pd.read_csv('train_v2.csv',nrows=5)
pre_df

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,hits,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",3162355547410993243,"{""continent"": ""Europe"", ""subContinent"": ""Weste...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508198450,1,1508198450
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8934116514970143966,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/transportation...",1508176307,6,1508176307
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7992466427990357681,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1508201613,1,1508201613
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9075655783635761930,"{""continent"": ""Asia"", ""subContinent"": ""Western...","[{'hitNumber': '1', 'time': '0', 'hour': '9', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508169851,1,1508169851
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6960673291025684308,"{""continent"": ""Americas"", ""subContinent"": ""Cen...","[{'hitNumber': '1', 'time': '0', 'hour': '14',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508190552,1,1508190552


**json**
- json형식이 문자열로 저장되어 있어서 변환작업이 필요
- 하나의 셀에 json형식이 들어가 있는 셀이 있음(따로 처리)
    - json이 한 겹인 컬럼 : device, totals, trafficSource
    - json이 두 겹인 컬럼 : customDimensions, hits
- 대용량데이터(25GB)의 원인으로 hits컬럼을 꼽았으며 hits 컬럼은 따로 핸들링하기로 결정

In [ ]:
# json 한 겹
pre_df['device'][0]

'{"browser": "Firefox", "browserVersion": "not available in demo dataset", "browserSize": "not available in demo dataset", "operatingSystem": "Windows", "operatingSystemVersion": "not available in demo dataset", "isMobile": false, "mobileDeviceBranding": "not available in demo dataset", "mobileDeviceModel": "not available in demo dataset", "mobileInputSelector": "not available in demo dataset", "mobileDeviceInfo": "not available in demo dataset", "mobileDeviceMarketingName": "not available in demo dataset", "flashVersion": "not available in demo dataset", "language": "not available in demo dataset", "screenColors": "not available in demo dataset", "screenResolution": "not available in demo dataset", "deviceCategory": "desktop"}'

In [ ]:
len(pre_df['device'][0])

736

In [ ]:
# json 두 겹
pre_df['hits'][0]

"[{'hitNumber': '1', 'time': '0', 'hour': '17', 'minute': '0', 'isInteraction': True, 'isEntrance': True, 'isExit': True, 'referer': 'https://www.google.co.uk/search?q=water+bottle&ie=utf-8&num=100&oe=utf-8&hl=en&gl=GB&uule=w+CAIQIFISCamRx0IRO1oCEXoliDJDoPjE&glp=1&gws_rd=cr&fg=1', 'page': {'pagePath': '/google+redesign/bags/water+bottles+and+tumblers', 'hostname': 'shop.googlemerchandisestore.com', 'pageTitle': 'Water Bottles & Tumblers | Drinkware | Google Merchandise Store', 'pagePathLevel1': '/google+redesign/', 'pagePathLevel2': '/bags/', 'pagePathLevel3': '/water+bottles+and+tumblers', 'pagePathLevel4': ''}, 'transaction': {'currencyCode': 'USD'}, 'item': {'currencyCode': 'USD'}, 'appInfo': {'screenName': 'shop.googlemerchandisestore.com/google+redesign/bags/water+bottles+and+tumblers', 'landingScreenName': 'shop.googlemerchandisestore.com/google+redesign/bags/water+bottles+and+tumblers', 'exitScreenName': 'shop.googlemerchandisestore.com/google+redesign/bags/water+bottles+and+tum

In [ ]:
len(pre_df['hits'][0])

3398

**데이터 불러오기**

In [ ]:
# hits를 제외한 컬럼 목록
col_list = pre_df.columns.tolist()
col_list.remove('hits')

# json이 포함된 목록(json 한겹)
json_columns = ['device', 'geoNetwork', 'totals', 'trafficSource']

# hits를 제외한 데이터 불러오기
df = pd.read_csv('train_v2.csv',
                 usecols=col_list,
                 dtype={'fullVisitorId' : 'str'},
                 converters={column : json.loads for column in json_columns},
                 parse_dates=['date'])

# unix timestamp를 datetime으로 변환
df['visitId'] = pd.to_datetime(df['visitId'], unit='s')
df['visitStartTime'] = pd.to_datetime(df['visitStartTime'], unit='s')

df.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",2017-10-16,"{'browser': 'Firefox', 'browserVersion': 'not ...",3162355547410993243,"{'continent': 'Europe', 'subContinent': 'Weste...",Not Socially Engaged,"{'visits': '1', 'hits': '1', 'pageviews': '1',...","{'campaign': '(not set)', 'source': 'google', ...",2017-10-17 00:00:50,1,2017-10-17 00:00:50
1,Referral,"[{'index': '4', 'value': 'North America'}]",2017-10-16,"{'browser': 'Chrome', 'browserVersion': 'not a...",8934116514970143966,"{'continent': 'Americas', 'subContinent': 'Nor...",Not Socially Engaged,"{'visits': '1', 'hits': '2', 'pageviews': '2',...",{'referralPath': '/a/google.com/transportation...,2017-10-16 17:51:47,6,2017-10-16 17:51:47
2,Direct,"[{'index': '4', 'value': 'North America'}]",2017-10-16,"{'browser': 'Chrome', 'browserVersion': 'not a...",7992466427990357681,"{'continent': 'Americas', 'subContinent': 'Nor...",Not Socially Engaged,"{'visits': '1', 'hits': '2', 'pageviews': '2',...","{'campaign': '(not set)', 'source': '(direct)'...",2017-10-17 00:53:33,1,2017-10-17 00:53:33
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",2017-10-16,"{'browser': 'Chrome', 'browserVersion': 'not a...",9075655783635761930,"{'continent': 'Asia', 'subContinent': 'Western...",Not Socially Engaged,"{'visits': '1', 'hits': '2', 'pageviews': '2',...","{'campaign': '(not set)', 'source': 'google', ...",2017-10-16 16:04:11,1,2017-10-16 16:04:11
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",2017-10-16,"{'browser': 'Chrome', 'browserVersion': 'not a...",6960673291025684308,"{'continent': 'Americas', 'subContinent': 'Cen...",Not Socially Engaged,"{'visits': '1', 'hits': '2', 'pageviews': '2',...","{'campaign': '(not set)', 'source': 'google', ...",2017-10-16 21:49:12,1,2017-10-16 21:49:12


**json to DataFrame**
-  json을 column=key, value=value인 DataFrame으로 만들어서 오른쪽에 붙여주는 작업

In [ ]:
json_columns.append('customDimensions') # json컬럼에 추가해서 한 번에 처리

for column in tqdm(json_columns):
    if column != 'customDimensions': # json이 한겹인 경우
        json_to_df = json_normalize(df[column])
        json_to_df.columns = [f"{column}.{subcol}" for subcol in json_to_df.columns] # 중복을 방지하기 위해 어느 컬럼의 json인지 기록
        df = df.drop(column, axis=1).merge(json_to_df, left_index=True, right_index=True) # json컬럼 지우고 변환된 df를 오른쪽에 붙여준다.
    else:
        # json 두 겹인 'customDimensions'는 converter가 적용되지 않아서 따로 작업을 진행
        tqdm.pandas() 
        df['customDimensions'] = df['customDimensions'].progress_apply(lambda x : ast.literal_eval(x)) # ""을 제거

        # json 한 겹([]) 풀어주기(0이라는 이름의 컬럼에 들어감)
        json_col2 = json_normalize(df['customDimensions'])

        # json을 column=key, value=value인 DataFrame으로 만들어줌(json_to_df와 같은 작업)
        json_to_df2 = json_normalize(json_col2[0])
        json_to_df2.columns = [f"{column}.{subcol}" for subcol in json_to_df2.columns]
        df = df.drop(column, axis=1).merge(json_to_df2, left_index=True, right_index=True)

df.head()

100%|██████████| 5/5 [14:00<00:00, 168.15s/it]


,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,device.browserSize,device.operatingSystem,device.operatingSystemVersion,device.isMobile,device.mobileDeviceBranding,device.mobileDeviceModel,device.mobileInputSelector,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.flashVersion,device.language,device.screenColors,device.screenResolution,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.cityId,geoNetwork.networkDomain,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.networkLocation,totals.visits,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.sessionQualityDim,totals.timeOnSite,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.referralPath,trafficSource.isTrueDirect,trafficSource.adContent,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.campaignCode,customDimensions.index,customDimensions.value
0,Organic Search,2017-10-16,3162355547410993243,Not Socially Engaged,2017-10-17 00:00:50,1,2017-10-17 00:00:50,Firefox,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Europe,Western Europe,Germany,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,1,1,1,NaN,NaN,NaN,NaN,(not set),google,organic,water bottle,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,EMEA
1,Referral,2017-10-16,8934116514970143966,Not Socially Engaged,2017-10-16 17:51:47,6,2017-10-16 17:51:47,Chrome,not available in demo dataset,not available in demo dataset,Chrome OS,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Cupertino,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,1,2,2,NaN,NaN,2,28,NaN,NaN,NaN,(not set),sites.google.com,referral,NaN,not available in demo dataset,/a/google.com/transportation/mtv-services/bike...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,North America
2,Direct,2017-10-16,7992466427990357681,Not Socially Engaged,2017-10-17 00:53:33,1,2017-10-17 00:53:33,Chrome,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,windjammercable.net,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,2,2,NaN,1,1,38,NaN,NaN,NaN,(not set),(direct),(none),NaN,not availab

**Unix to Datetime**
- visitId 와 visitStratTime컬럼이 unix시간으로 되어있어서 변환
- 두 컬럼의 값이 거의 동일(몇 초 차이)
- 방문시간 확인용으로 사용하기 위해 visitStartTime를 남김
- fullVisitorId가 고유아이디이기 때문에 visitId컬럼은 제거

In [ ]:
# 큰 차이가 없는 것으로 보임
# diff_time = set(df['visitId']).difference(set(df['visitStartTime']))
df[df['visitId'].isin(diff_time)].loc[:,['visitId','visitStartTime']].head()

,visitId,visitStartTime
56,2017-10-16 21:45:56,2017-10-16 21:46:00
569,2017-10-16 16:43:43,2017-10-16 16:43:52
1731,2017-10-16 23:18:55,2017-10-16 23:19:01
1853,2017-10-16 17:33:50,2017-10-16 17:33:53
2153,2017-10-16 23:32:03,2017-10-16 23:32:06


In [ ]:
# 고유id는 fullVisitorId가 존재하여 visitId 제외
df = df.drop('visitId',axis=1)

**단일 값인 컬럼**
- 다수의 컬럼에서 수집되지 않은 컬럼이 존재
- 단일 값인 컬럼 제거

In [ ]:
df['socialEngagementType'].unique()

array(['Not Socially Engaged'], dtype=object)

In [ ]:
df['device.browserVersion'].unique() 

array(['not available in demo dataset'], dtype=object)

In [ ]:
df['device.browserSize'].unique() 

array(['not available in demo dataset'], dtype=object)

In [ ]:
just_one_value = [column for column in df.columns if len(df[column].unique())==1]
just_one_value

['socialEngagementType',
 'device.browserVersion',
 'device.browserSize',
 'device.operatingSystemVersion',
 'device.mobileDeviceBranding',
 'device.mobileDeviceModel',
 'device.mobileInputSelector',
 'device.mobileDeviceInfo',
 'device.mobileDeviceMarketingName',
 'device.flashVersion',
 'device.language',
 'device.screenColors',
 'device.screenResolution',
 'geoNetwork.cityId',
 'geoNetwork.latitude',
 'geoNetwork.longitude',
 'geoNetwork.networkLocation',
 'totals.visits',
 'trafficSource.adwordsClickInfo.criteriaParameters']

In [ ]:
df.shape

(1708337, 60)

In [ ]:
# 단일 값 컬럼 제거
df = df.drop(just_one_value, axis=1)
df.shape

(1708337, 41)

**값이 두개 뿐인 컬럼**

In [ ]:
# 비디오광고를 클릭하고 들어온 경우(비디오광고 없음)
df['trafficSource.adwordsClickInfo.isVideoAd'].value_counts()

False    75274
Name: trafficSource.adwordsClickInfo.isVideoAd, dtype: int64

In [ ]:
# 1개를 제외하고 모두 결측치
df['trafficSource.campaignCode'].isna().sum()

1708336

In [ ]:
# 값이 두개인 컬럼 제거
df = df.drop(['trafficSource.adwordsClickInfo.isVideoAd','trafficSource.campaignCode'],axis=1)

**다른 지표로 판단가능한 지표**
- device.isMobile : 모바일여부 (device.deviceCategory로 기기 확인 가능하여 제거)
- trafficSource.isTrueDirect : 직접 방문여부 (trafficSource.source와 direct 여부가 다르게 나타나서 신빙성 없는 컬럼으로 보고 제거)

In [ ]:
# 모바일, 태블릿 확인가능
df['device.deviceCategory'].value_counts()

desktop    1171579
mobile      471336
tablet       65422
Name: device.deviceCategory, dtype: int64

In [ ]:
# (direct)값이 존재
df['trafficSource.source'].value_counts()

google                  658384
(direct)                565975
youtube.com             329450
analytics.google.com     37436
Partners                 32931
                         ...  
google.ge                    1
apps.rackspace.com           1
blackboard.neu.edu           1
google.se                    1
p.qiao.baidu.com             1
Name: trafficSource.source, Length: 345, dtype: int64

In [ ]:
# 데이터 정합성이 의심되어 삭제처리
df = df.drop(['device.isMobile','trafficSource.isTrueDirect','trafficSource.referralPath'], axis=1)

**사용자 지정 컬럼(보류)**

In [ ]:
# 사용자 지정 차원이 4와 nan으로 나뉨
df[df['customDimensions.index']=='4']['customDimensions.value'].value_counts()

North America      768223
EMEA               313991
APAC               222071
South America       45553
Central America     25264
Name: customDimensions.value, dtype: int64

In [ ]:
# 4가 아닌 값은 모두 nan
df[df['customDimensions.index']!='4']['customDimensions.value'].value_counts()

Series([], Name: customDimensions.value, dtype: int64)

**컬럼순서를 사용성 있게 변경**

In [ ]:
# 순서변경
last_df = df[['fullVisitorId','date','visitStartTime',        # 유저고유ID, 방문날짜, 세션 방문 시간
    'totals.timeOnSite','visitNumber', # totals.timeOnSite: 해당사이트에 머문시간(초), visitNumber: 몇 번째 방문 인지
    'channelGrouping','device.browser','device.operatingSystem','device.deviceCategory', # 유입매체, 브라우저, 운영체제, 기기
    'geoNetwork.continent','geoNetwork.subContinent', 'geoNetwork.country','geoNetwork.region','geoNetwork.metro','geoNetwork.city','geoNetwork.networkDomain', # 지역
    'totals.hits', 'totals.pageviews', 'totals.bounces','totals.newVisits', # 'bounces' = 다른페이지 방문 없이 이탈여부, 'newVisits' = 재방문여부 
    'totals.sessionQualityDim','totals.transactions','totals.transactionRevenue','totals.totalTransactionRevenue', # qualitydim =활동점수, transaction=구매활동횟수,transactionRevenue=비용제거거래액,totals.totalTransactionRevenue=비용포함
    'trafficSource.campaign', 'trafficSource.adContent','trafficSource.source', # campaign=캠페인, adContent=캠패인 안의 컨텐츠, source=유입사이트
    'trafficSource.medium','trafficSource.keyword',      # medium=유입매체, keyword=유입검색단어,
    'trafficSource.adwordsClickInfo.page','trafficSource.adwordsClickInfo.slot', # adwordsClickInfo.page=광고유입 후 방문한페이지수, adwordsClickInfo.slot=광고위치("RHS"는 "right-hand side"오른쪽, "Google Display Network"는 검색결과창 외의 위치)
    'trafficSource.adwordsClickInfo.gclId','trafficSource.adwordsClickInfo.adNetworkType', # gclId=구글ad로 유입(아닌 경우Nan), adNetworkType=유입광고위치 googlesearch(검색결과창),GDN(타사이트 광고창)
    'customDimensions.index','customDimensions.value' # 사용자 지정 차원, 값(4차원엔 값이 있고, 결측차원엔 값이 없음)
    ]]
# channelGrouping 컬럼
# "Organic Search" (검색 엔진), "Direct" (직접 방문), "Referral" (외부 웹사이트에서의 링크), "Paid Search" (광고), 
# "Social" (소셜 미디어), "Email" (이메일), "Affiliates" (제휴사), "Display" (디스플레이 광고)

# trafficsource.medium
# organic(웹사이트검색),referral(링크유입), affiliate(제휴사), cpc,cpm(클릭당,조회수당 비용측정)

last_df.shape

(1708337, 36)

In [ ]:
# 빠진 컬럼 없는지 확인
len(set(df.columns).intersection(set(last_df.columns)))

36

In [ ]:
# 시간순 정렬
last_df = last_df.sort_values(by=['visitStartTime'],ascending=True)
last_df.head()

,fullVisitorId,date,visitStartTime,totals.timeOnSite,visitNumber,channelGrouping,device.browser,device.operatingSystem,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.networkDomain,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.sessionQualityDim,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue,trafficSource.campaign,trafficSource.adContent,trafficSource.source,trafficSource.referralPath,trafficSource.medium,trafficSource.keyword,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,customDimensions.index,customDimensions.value
997408,0423043652415339154,2016-08-01,2016-08-01 07:00:12,NaN,3,Direct,Safari,iOS,mobile,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),1,1.0,1.0,NaN,NaN,NaN,NaN,NaN,(not set),NaN,(direct),NaN,(none),NaN,NaN,NaN,NaN,NaN,4.0,North America
996613,8294721032567046680,2016-08-01,2016-08-01 07:04:26,NaN,1,Social,Chrome,Windows,desktop,Asia,Southeast Asia,Thailand,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown,1,1.0,1.0,1.0,NaN,NaN,NaN,NaN,(not set),NaN,youtube.com,/yt/about/,referral,NaN,NaN,NaN,NaN,NaN,NaN,NaN
995982,7718623669497357235,2016-08-01,2016-08-01 07:04:41,40.0,1,Organic Search,Amazon Silk,Android,tablet,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),5,5.0,NaN,1.0,NaN,NaN,NaN,NaN,(not set),NaN,google,NaN,organic,(not provided),NaN,NaN,NaN,NaN,4.0,North America
996177,4798058133221713505,2016-08-01,2016-08-01 07:06:01,89.0,1,Organic Search,Chrome,Windows,desktop,Americas,Northern America,Canada,not available in demo dataset,not available in demo dataset,not available in demo dataset,bell.ca,9,7.0,NaN,1.0,NaN,NaN,NaN,NaN,(not set),NaN,google,NaN,organic,(not provided),NaN,NaN,NaN,NaN,4.0,North America
997143,5076747981380011349,2016-08-01,2016-08-01 07:06:10,NaN,1,Direct,Chrome,Windows,desktop,Asia,Southeast Asia,Philippines,Metro Manila,(not set),Quezon City,pldt.net,1,1.0,1.0,1.0,NaN,NaN,NaN,NaN,(not set),NaN,(direct),NaN,(none),NaN,NaN,NaN,NaN,NaN,NaN,NaN


**결측치 제거**

In [ ]:
# 결측치가 있는 컬럼 목록 만들기
nul_col = []

def col_isnul(col):
    if last_df[col].isnull().sum() != 0:
        nul_col.append(col)
        print(f'{col} : {last_df[col].isna().sum()}')

In [ ]:
for i in last_df.columns:
    col_isnul(i)

totals.timeOnSite : 874294
totals.pageviews : 239
totals.bounces : 836759
totals.newVisits : 400907
totals.sessionQualityDim : 835274
totals.transactions : 1689778
totals.transactionRevenue : 1689823
totals.totalTransactionRevenue : 1689823
trafficSource.adContent : 1643600
trafficSource.referralPath : 1142073
trafficSource.keyword : 1052780
trafficSource.adwordsClickInfo.page : 1633063
trafficSource.adwordsClickInfo.slot : 1633063
trafficSource.adwordsClickInfo.gclId : 1632914
trafficSource.adwordsClickInfo.adNetworkType : 1633063
customDimensions.index : 333235
customDimensions.value : 333235


In [ ]:
# 수치형 데이터들은 결측치를 0으로 채워줌
# totals.timeOnSite / totals.pageviews / totals.bounces / totals.newVisits / totals.sessionQualityDim
# totals.transactions / totals.transactionRevenue / totals.totalTransactionRevenue / 
last_df[nul_col[0:8]] = last_df[nul_col[0:8]].fillna(0)
last_df[nul_col[0:8]].isnull().sum()

totals.timeOnSite                 0
totals.pageviews                  0
totals.bounces                    0
totals.newVisits                  0
totals.sessionQualityDim          0
totals.transactions               0
totals.transactionRevenue         0
totals.totalTransactionRevenue    0
dtype: int64

In [ ]:
# 결측치인데, 다르게 표시된 값들 결측치로 변경
# (not set), (not available in demo dataset), (not provide), (none), unknown.unknown

not_set = ['geoNetwork.continent','geoNetwork.subContinent','geoNetwork.subContinent','geoNetwork.region','geoNetwork.country',
           'geoNetwork.metro','geoNetwork.city','geoNetwork.networkDomain','device.operatingSystem','trafficSource.campaign','trafficSource.medium',]
not_ava = ['geoNetwork.region','geoNetwork.metro','geoNetwork.city']

for col in not_set:
    last_df[col].replace('(not set)', np.nan, inplace=True)

for col in not_ava:
    last_df[col].replace('not available in demo dataset', np.nan, inplace=True)

last_df['geoNetwork.networkDomain'].replace('unknown.unknown', np.nan, inplace=True)
last_df['trafficSource.medium'].replace('(none)',np.nan, inplace=True)
last_df['trafficSource.keyword'].replace('(not provided)',np.nan, inplace=True)

**데이터 정합성 체크 (진짜 Direct 유저와 미식별 유저 분리)**

In [ ]:
last_df.head()

,fullVisitorId,date,visitStartTime,totals.timeOnSite,visitNumber,channelGrouping,device.browser,device.operatingSystem,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.networkDomain,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.sessionQualityDim,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue,trafficSource.campaign,trafficSource.adContent,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,customDimensions.index,customDimensions.value
997408,0423043652415339154,2016-08-01,2016-08-01 07:00:12,0.0,3,Direct,Safari,iOS,mobile,Americas,Northern America,United States,NaN,NaN,NaN,NaN,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,(direct),NaN,NaN,NaN,NaN,NaN,NaN,4.0,North America
996613,8294721032567046680,2016-08-01,2016-08-01 07:04:26,0.0,1,Social,Chrome,Windows,desktop,Asia,Southeast Asia,Thailand,NaN,NaN,NaN,NaN,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,youtube.com,referral,NaN,NaN,NaN,NaN,NaN,NaN,NaN
995982,7718623669497357235,2016-08-01,2016-08-01 07:04:41,40.0,1,Organic Search,Amazon Silk,Android,tablet,Americas,Northern America,United States,NaN,NaN,NaN,NaN,5,5.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,google,organic,NaN,NaN,NaN,NaN,NaN,4.0,North America
996177,4798058133221713505,2016-08-01,2016-08-01 07:06:01,89.0,1,Organic Search,Chrome,Windows,desktop,Americas,Northern America,Canada,NaN,NaN,NaN,bell.ca,9,7.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,google,organic,NaN,NaN,NaN,NaN,NaN,4.0,North America
997143,5076747981380011349,2016-08-01,2016-08-01 07:06:10,0.0,1,Direct,Chrome,Windows,desktop,Asia,Southeast Asia,Philippines,Metro Manila,NaN,Quezon City,pldt.net,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,(direct),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# channelGrouping의 organic이 잘 분류된 것을 보고 direct도 channelGrouping를 기준으로 정합성을 체크하기로 결정
last_df[(last_df['trafficSource.medium']=='organic')]['channelGrouping'].value_counts()

Organic Search    591783
Name: channelGrouping, dtype: int64

In [ ]:
# 확인 결과 channelGrouping의 절반이 direct가 아닌 것으로 확인
last_df[(last_df['trafficSource.source']=='(direct)')]['channelGrouping'].value_counts()

Direct            273134
Organic Search    147180
Referral          133315
Paid Search        12247
Display               96
(Other)                3
Name: channelGrouping, dtype: int64

In [ ]:
# 둘의 값이 direct가 아닌 값은 unidentified로 변경 
unidentified = last_df[(last_df['trafficSource.source']=='(direct)')&(last_df['channelGrouping']!='Direct')].index
last_df.loc[unidentified,'trafficSource.source'] = 'unidentified'

last_df[(last_df['trafficSource.source']=='(direct)')]['channelGrouping'].value_counts()

Direct    273134
Name: channelGrouping, dtype: int64

In [ ]:
last_df['trafficSource.source'].value_counts()

google                      658384
youtube.com                 329450
unidentified                292841
(direct)                    273134
analytics.google.com         37436
                             ...  
msn                              1
google.no                        1
good.barkpost.com                1
se.yhs4.search.yahoo.com         1
m.wikihow.com                    1
Name: trafficSource.source, Length: 346, dtype: int64

In [ ]:
# 중간저장
last_df.to_csv('without_hits.csv', index=False)